_We're going to be using pandas an json, so lets start with that._

In [15]:
import json
import pandas as pd
from pandas.io.json import json_normalize

In [42]:
# simple snippet for discussing string concatention. Remove this.

topic_name = "karunopenweather"
base_filename = "_weather.json"

full_filename = topic_name + base_filename
print (full_filename)

karunopenweather_weather.json


_Before we do any real work, can we list the files from our target directory?_

In [35]:
!pwd
! ls ../APIs

/home/vagrant/git/BigData/democratizing_weather_data/democratizing_weather_data/streaming/prototypes
API_index.csv	     wsdot.TrafficFlows.json  yahoo_weather.json
openweathermap.json  wsdot.Traveltimes.json


_Nice! Let's examine a json file_

In [43]:
!head ../APIs/yahoo_weather.json

{
    "query": {
        "count": 1,
        "created": "2017-07-29T23:51:25Z",
        "lang": "en-US",
        "diagnostics": {
            "publiclyCallable": "true",
            "url": {
                "execution-start-time": "1",
                "execution-stop-time": "5",


_Read a json file from disk (we'll use ../API/<tbd>.json
We should reall use the yahoo api output, since we know how to convert that to a csv_

Source [Andy Hayden on ghthub.io](http://hayd.github.io/2013/pandas-json)

In [23]:
df = pd.read_json('../APIs/yahoo_weather.json')
df

,query
count,1
created,2017-07-29T23:51:25Z
diagnostics,"{'publiclyCallable': 'true', 'url': {'executio..."
lang,en-US
results,"{'channel': {'units': {'distance': 'mi', 'pres..."


_Not quite what we wanted.  Let's read then normalize_ 

Read file with json Library. Source: [Scott Robinson's Stack Abuse Article ](http://stackabuse.com/reading-and-writing-json-to-a-file-in-python/)

In [64]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

raw_json

normal_json = json_normalize(raw_json['query']) 
normal_json

,count,created,diagnostics.build-version,diagnostics.publiclyCallable,diagnostics.service-time,diagnostics.url.content,diagnostics.url.execution-start-time,diagnostics.url.execution-stop-time,diagnostics.url.execution-time,diagnostics.user-time,...,results.channel.location.region,results.channel.title,results.channel.ttl,results.channel.units.distance,results.channel.units.pressure,results.channel.units.speed,results.channel.units.temperature,results.channel.wind.chill,results.channel.wind.direction,results.channel.wind.speed
0,1,2017-07-29T23:51:25Z,2.0.164,true,4,http://weather-ydn-yql.media.yahoo.com:4080/v3...,1,5,4,6,...,CA,"Yahoo! Weather - Sunnyvale, CA, US",60,mi,in,mph,F,75,355,18


In [65]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

raw_json

normal_json = json_normalize(raw_json['query']['results']) 
normal_json

,channel.astronomy.sunrise,channel.astronomy.sunset,channel.atmosphere.humidity,channel.atmosphere.pressure,channel.atmosphere.rising,channel.atmosphere.visibility,channel.description,channel.image.height,channel.image.link,channel.image.title,...,channel.location.region,channel.title,channel.ttl,channel.units.distance,channel.units.pressure,channel.units.speed,channel.units.temperature,channel.wind.chill,channel.wind.direction,channel.wind.speed
0,6:10 am,8:19 pm,58,1006.0,0,16.1,"Yahoo! Weather for Sunnyvale, CA, US",18,http://weather.yahoo.com,Yahoo! Weather,...,CA,"Yahoo! Weather - Sunnyvale, CA, US",60,mi,in,mph,F,75,355,18


In [66]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

raw_json

normal_json = json_normalize(raw_json['query']['results']['channel']) 
normal_json

,astronomy.sunrise,astronomy.sunset,atmosphere.humidity,atmosphere.pressure,atmosphere.rising,atmosphere.visibility,description,image.height,image.link,image.title,...,location.region,title,ttl,units.distance,units.pressure,units.speed,units.temperature,wind.chill,wind.direction,wind.speed
0,6:10 am,8:19 pm,58,1006.0,0,16.1,"Yahoo! Weather for Sunnyvale, CA, US",18,http://weather.yahoo.com,Yahoo! Weather,...,CA,"Yahoo! Weather - Sunnyvale, CA, US",60,mi,in,mph,F,75,355,18


In [67]:
with open('../APIs/yahoo_weather.json') as json_file:
  raw_json = json.load(json_file)

raw_json

normal_json = json_normalize(raw_json['query']['results']['channel']['astronomy']) 
normal_json

,sunrise,sunset
0,6:10 am,8:19 pm


In [46]:
# I don't think I really need to do this. Prettu sure the output of json.load() is already a dataframe
df = normal_json
df

,astronomy.sunrise,astronomy.sunset,atmosphere.humidity,atmosphere.pressure,atmosphere.rising,atmosphere.visibility,description,image.height,image.link,image.title,...,location.region,title,ttl,units.distance,units.pressure,units.speed,units.temperature,wind.chill,wind.direction,wind.speed
0,6:10 am,8:19 pm,58,1006.0,0,16.1,"Yahoo! Weather for Sunnyvale, CA, US",18,http://weather.yahoo.com,Yahoo! Weather,...,CA,"Yahoo! Weather - Sunnyvale, CA, US",60,mi,in,mph,F,75,355,18


## Let's try another json file (openweathermap.json)

In [51]:
!pwd
! ls ../APIs
!head ../APIs/openweathermap.json

/home/vagrant/git/BigData/democratizing_weather_data/democratizing_weather_data/streaming/prototypes
API_index.csv	     wsdot.TrafficFlows.json  yahoo_weather.json
openweathermap.json  wsdot.Traveltimes.json
{
    "coord": {
        "lon": 139.01,
        "lat": 35.02
    },
    "weather": [
        {
            "id": 800,
            "main": "Clear",
            "description": "clear sky",


In [62]:
with open('../APIs/openweathermap.json') as json_file:
  raw_json = json.load(json_file)

#let's normalize, with no field-seeking.
normal_json = json_normalize(raw_json) 
normal_json                              

,base,clouds.all,cod,coord.lat,coord.lon,dt,id,main.grnd_level,main.humidity,main.pressure,...,main.temp_max,main.temp_min,name,sys.country,sys.message,sys.sunrise,sys.sunset,weather,wind.deg,wind.speed
0,stations,0,200,35.02,139.01,1485792967,1907296,1013.75,100,1013.75,...,285.514,285.514,Tawarano,JP,0.0025,1485726240,1485763863,"[{'id': 800, 'main': 'Clear', 'description': '...",311,5.52


_Okay, Pretty flat and clean, except the wether field. Let's nromalize that one_

In [61]:
weather_json = json_normalize(raw_json['weather'])
weather_json

,description,icon,id,main
0,clear sky,01n,800,Clear


_Not bad.  Next step would be splice this into the un-normalized record, and replace that weather field_

## Future Work
- Use Postman to generate more yahoo weather json files, then cocatenate them into one dataframe
- Try other json files
- Get to the point where you have multiple rows to play with. 